<a href="https://colab.research.google.com/github/inderpreetsingh01/PyTorch/blob/main/D2l_2_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [19]:
def create_data(w=torch.tensor([2, -3.4]), b=4.2, noise=0.01, num_train=1000, num_val=1000):
  n = num_train + num_val
  X = torch.randn(n, len(w))
  noise = torch.randn(n, 1) * noise
  y = torch.matmul(X, w.reshape((-1, 1))) + b + noise
  return X, y

In [20]:
X, y = create_data()

In [21]:
X.shape

torch.Size([2000, 2])

In [22]:
y.shape

torch.Size([2000, 1])

In [23]:
n_train = 1000
batch_size = 32

X_train = X[slice(0, n_train)]
y_train = y[slice(0, n_train)]

X_test = X[slice(n_train, None)]
y_test = y[slice(n_train, None)]

train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

In [24]:
X, y = next(iter(train_dataloader))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


In [25]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [26]:
class LinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(2, 1)
    self.linear.weight.data.normal_(0, 0.01)
    self.linear.bias.data.fill_(0.0)

  def forward(self, x):
    return self.linear(x)

In [27]:
model = LinearRegression().to(device)
loss_fn = nn.MSELoss(reduction='mean')
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# applying L2 regularization on weights only
optimizer = torch.optim.SGD([
            {'params': model.linear.weight, 'weight_decay': 1e-2},
            {'params': model.linear.bias}], lr=1e-3)

In [28]:
def train(dataloader, model, loss_fn, optimizer, batch_size):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    X = X.to(device)
    y = y.to(device)

    y_pred = model(X)
    loss = loss_fn(y, y_pred)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # print(f"{batch*batch_size}/{size}")

In [29]:
def test(dataloader, model, loss_fn, batch_size):
  model.eval()
  total_loss = 0
  # size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.to(device)
    y = y.to(device)
    with torch.no_grad():
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      total_loss += loss.item()

  print(f"Loss: {total_loss/len(dataloader)}")

In [30]:
for epoch in range(10):
  print(f"\nEpoch: {epoch+1}")
  train(train_dataloader, model, loss_fn, optimizer, batch_size)
  test(test_dataloader, model, loss_fn, batch_size)


Epoch: 1
Loss: 28.01163613796234

Epoch: 2
Loss: 24.773563623428345

Epoch: 3
Loss: 21.8898546397686

Epoch: 4
Loss: 19.35808601975441

Epoch: 5
Loss: 17.1186543405056

Epoch: 6
Loss: 15.136662244796753

Epoch: 7
Loss: 13.388215780258179

Epoch: 8
Loss: 11.811426624655724

Epoch: 9
Loss: 10.42312741279602

Epoch: 10
Loss: 9.21254786849022


In [31]:
model.linear.weight.data

tensor([[ 0.8812, -1.5653]])

In [32]:
model.linear.bias.data

tensor([1.9576])

In [33]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

linear.weight tensor([[ 0.8812, -1.5653]])
linear.bias tensor([1.9576])
